# Model Deployment and Invocation

In [9]:
import tarfile
import sagemaker
from sagemaker import get_execution_role
from sagemaker.tensorflow import TensorFlowModel
import numpy as np

## Creating Archive File for Saved Model

In [6]:
# Create a tar.gz file containing the model files
with tarfile.open('model.tar.gz', 'w:gz') as tar:
    tar.add('model/assets', arcname='1/assets')
    tar.add('model/variables', arcname='1/variables')
    tar.add('model/saved_model.pb', arcname='1/saved_model.pb')
    tar.add('model/fingerprint.pb', arcname='1/fingerprint.pb')

## Uploading Model to S3 Bucket

In [7]:
# Get the default Sagemaker session
sagemaker_session = sagemaker.Session()
role = get_execution_role()

model_path = 'model.tar.gz'
bucket_name = 'dsci6007-final-project'
s3_key = 'model'  # The S3 path where you want to upload the model

# Upload the model to S3
s3_model_path = sagemaker_session.upload_data(path=model_path, bucket=bucket_name, key_prefix=s3_key)

print("Model uploaded to:", s3_model_path)

Model uploaded to: s3://dsci6007-final-project/model/model.tar.gz


## Deploying Model as Sagemaker Endpoint

In [8]:
# Specify the S3 path where your model is stored
model_s3_path = 's3://dsci6007-final-project/model/model.tar.gz'

# Create a TensorFlowModel object
model = TensorFlowModel(model_data=model_s3_path, role=role, framework_version='2.14')

# Deploy the model as an endpoint
predictor = model.deploy(instance_type='ml.m5.xlarge', initial_instance_count=1)

----!

## Invoking the Endpoint

In [21]:
# Load model input and relevant info for predictions
input_data = np.load('input_data.npy')
max_cpi = 281.695
min_cpi = 269.819

In [22]:
# Generate prediction from the model
prediction = predictor.predict(input_data)['predictions'][0][0]

In [24]:
forecast =  max_cpi-prediction/(max_cpi-min_cpi)
print('The Core CPI value for the month Nov 2021 predicted by LSTM is ', round(forecast,3))

pct_chg = ((forecast - min_cpi)/min_cpi) * 100
print('The forecasted U.S. Core Consumer Price Index (CPI) YoY is ', round(pct_chg,2), '%')

The Core CPI value for the month Nov 2021 predicted by LSTM is  281.625
The forecasted U.S. Core Consumer Price Index (CPI) YoY is  4.38 %
